# Emerging Technolgoies

### Libraries 

In [ ]:
# Random selections.
import random

# Numerical arrays.
import numpy as np

## Problem 1: Generating Random Boolean Functions


In [ ]:
def f1(a):
  """Takes a single bit but always returns 0."""
  return 0

def f2(a):
  """Identity function."""
  return a

def f3(a):
  """Not function."""
  if a == 0:
    return 1
  else:
    return 0

def f4(a):
  """Constant 1 function."""
  return 1

## Problem 2: Classical Testing for Function Type


## Problem 3: Quantum Oracles


## Problem 4: Deutsch's Algorithm with Qiskit


## Problem 5: Scaling to the Deutsch–Jozsa Algorithm
